In [1]:
import numpy as np

In [2]:
X = np.array([[1,2,3],[4,5,6],[7,8,9]])
np.max(X)

9

In [3]:
import torch

/home/matt/miniconda3/envs/mmdet-solaris/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
torch.sign(torch.rand(1) - 0.5)

tensor([-1.])